In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 43)                731       
                                                                 
Total params: 17,275
Trainable params: 17,275
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
12/12 [==============================] - 4s 6ms/step - loss: 3.7588 - accuracy: 0.0294
Epoch 2/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7528 - accuracy: 0.0481
Epoch 3/550
12/12 [==============================] - 0s 7ms/step - loss: 3.7466 - accuracy: 0.0588
Epoch 4/550
12/12 [==============================] - 0s 8ms/step - loss: 3.7384 - accuracy: 0.0588
Epoch 5/550
12/12 [==============================] - 0s 7ms/step - loss: 3.7286 - accuracy: 0.0588
Epoch 6/550
12/12 [==============================] - 0s 9ms/step - loss: 3.7174 - accuracy: 0.0588
Epoch 7/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7032 - accuracy: 0.0588
Epoch 8/550
12/12 [==============================] - 0s 9ms/step - loss: 3.6870 - accuracy: 0.0588
Epoch 9/550
12/12 [==============================] - 0s 8ms/step - loss: 3.6696 - accuracy: 0.0588
Epoch 10/550
12/12 [==============================] - 0s 9ms/step - loss: 3.6524 - accuracy: 0.0588
Epoch 11/

In [9]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
